# Ex 1-1 by Chainer
- https://docs.chainer.org/en/stable/examples/mnist.html
- https://github.com/kose/chainer-linear-regression/blob/master/net.py
- https://multithreaded.stitchfix.com/blog/2015/12/09/intro-to-chainer/

Chainer는 모델을 만들 때, 케라스나 파이토치하고 다른 형태로 만든다. 예제 1-1은 단일 노드로 구성된 단일 계층 네트웍을 다루고 있기 때문에 Linear(1,1)로 뉴럴넷을 모델링했다. 복수 계층을 가지는 경우는 다른 형태로 모델링을 구성해야 한다.

In [88]:
import chainer

import numpy as np

x = np.array([0, 1, 2, 3, 4]).astype('float32').reshape(-1,1) 
y = x * 2 + 1

model = chainer.links.Linear(1,1)
optimizer = chainer.optimizers.SGD()
optimizer.setup(model)

for _ in range(1000):
    output = model(x)
    loss = chainer.functions.mean_squared_error(y, output)
    model.zerograds()
    loss.backward()
    optimizer.update()

model(x)

variable([[0.999104 ],
          [2.9994187],
          [4.9997334],
          [7.000048 ],
          [9.000362 ]])